In [5]:
import pandas as pd
import MetaTrader5 as mt5
import talib as ta
from datetime import datetime,timedelta
from out_in_bb_backtest import setup_tsl,setup_stop_tp
import numpy as np
from backtesting import Strategy, Backtest
import warnings
warnings.filterwarnings("ignore")

if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

ativo = 'EURUSD'
#ativo = 'WIN@N'
df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M5, 0, 14000))
df['Data'] = pd.to_datetime(df['time'], unit='s').apply(lambda x: str(x))
df = df[['Data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df_puro = df.rename(columns = {'real_volume':'vol'})
df2 = df_puro.copy()

In [6]:
result = pd.read_excel('resultado_eurusd_5M_ema_False_stop_tp.xlsx')
result.sort_values(by = 'Taxa_acerto', ascending=False).iloc[0:30]

,Timeframe,pontos,Rate_tp,Time_period,var_bb,N_trades,Taxa_acerto,Pontos_total,Retorno
92,5,0.0015,1,20,2.2,294,59.863946,0.07230,4.785957
56,5,0.0005,1,20,2.2,344,58.430233,0.01180,2.846640
40,5,0.0015,1,14,2.0,385,57.402597,0.06625,3.494168
32,5,0.0010,1,14,2.2,420,57.142857,0.03900,4.097636
44,5,0.0015,1,14,2.2,350,57.142857,0.05750,3.636431
28,5,0.0010,1,14,2.0,466,56.437768,0.03670,3.525568
68,5,0.0008,1,20,2.2,333,56.156156,0.01615,3.272406
36,5,0.0015,1,14,1.8,404,55.693069,0.04880,1.200465
80,5,0.0010,1,20,2.2,321,55.451713,0.01895,2.664918
16,5,0.0008,1,14,2.0,496,55.443548,0.01840,3.032692


In [13]:
for index,result_atual in result.loc[[37,12]].iterrows():
    time_period = result_atual['Time_period']
    var_bb = result_atual['var_bb']
    pontos = result_atual['pontos']
    rate_stop = 1
    rate_tp = result_atual['Rate_tp']
    retorno = result_atual['Retorno']
    Pontos_total = result_atual['Pontos_total']
    spread = 0.00005
    timeframe = 5


    df=df_puro.copy()
    bt = setup_stop_tp(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=False)
    #bt = setup_tsl(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=False)

    stats = bt.run()

    df_trades = stats['_trades']
    df_trades['Hora_h'] = df_trades['EntryTime'].apply(lambda x: x.hour)
    df_trades['resultado_binario'] = df_trades['ReturnPct'].apply(lambda x: 1 if x > 0 else 0)

    print(f'Time_period: {time_period} -- var_bb: {var_bb} -- pontos: {pontos}--Rate_tp: {rate_tp}-- Retorno: {round(retorno,2)} -- Pontos_total: {Pontos_total}')
    df_new_hora = df_trades[['Hora_h','resultado_binario']]
    df_new_hora = df_trades.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

    #df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
    print(df_new_hora)


Time_period: 14.0 -- var_bb: 1.8 -- pontos: 0.0015--Rate_tp: 2.0-- Retorno: 6.93 -- Pontos_total: 0.2803999999999999
    Hora_h  qtd_operacoes  taxa_acerto
0        0              9     0.333333
1        1             14     0.500000
2        2             13     0.615385
3        3             16     0.437500
4        4             18     0.444444
5        5              7     0.571429
6        6             10     0.500000
7        7              5     0.200000
8        8             11     0.363636
9        9             16     0.562500
10      10             11     0.181818
11      11             19     0.473684
12      12             18     0.611111
13      13             19     0.631579
14      14             17     0.470588
15      15             13     0.230769
16      16             28     0.714286
17      17             19     0.421053
18      18              1     1.000000
19      19             22     0.500000
20      20             40     0.600000
21      21             11

In [5]:
bt.plot()

Row(id='1418', ...)

In [8]:
df_trades = stats['_trades']
df_trades['Hora_h'] = df_trades['EntryTime'].apply(lambda x: x.hour)
df_trades['resultado_binario'] = df_trades['ReturnPct'].apply(lambda x: 1 if x > 0 else 0)

print(f'Time_period: {time_period} -- var_bb: {var_bb} -- pontos: {pontos}--Rate_tp: {rate_tp}-- Retorno: {round(retorno,2)} -- Pontos_total: {Pontos_total}')
df_new_hora = df_trades[['Hora_h','resultado_binario']]
df_new_hora = df_trades.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

#df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)

Time_period: 14.0 -- var_bb: 1.8 -- pontos: 0.0015--Rate_tp: 2.0-- Retorno: 6.93 -- Pontos_total: 0.2803999999999999
    Hora_h  qtd_operacoes  taxa_acerto
0        0              9     0.333333
1        1             14     0.500000
2        2             13     0.615385
3        3             16     0.437500
4        4             18     0.444444
5        5              7     0.571429
6        6             10     0.500000
7        7              5     0.200000
8        8             11     0.363636
9        9             16     0.562500
10      10             11     0.181818
11      11             19     0.473684
12      12             18     0.611111
13      13             19     0.631579
14      14             17     0.470588
15      15             13     0.230769
16      16             28     0.714286
17      17             19     0.421053
18      18              1     1.000000
19      19             22     0.500000
20      20             40     0.600000
21      21             11

#### Setup_Tsl

In [34]:
result = pd.read_excel('resultado_eurusd_5M_ema_False_tsl.xlsx')
result.sort_values(by = 'Retorno', ascending=False).iloc[:]

,Timeframe,pontos,Time_period,var_bb,N_trades,Taxa_acerto,Retorno
0,5,0.001,14,1.8,490,42.857143,4.401683
1,5,0.001,14,2.0,467,46.252677,2.994024
8,5,0.001,30,2.2,236,47.033898,2.719672
7,5,0.001,30,2.0,292,48.287671,1.914844
2,5,0.001,14,2.2,409,47.677262,1.757775
5,5,0.001,20,2.2,320,46.562500,1.417995
6,5,0.001,30,1.8,328,41.463415,0.933763
4,5,0.001,20,2.0,387,44.702842,0.907120
3,5,0.001,20,1.8,372,43.817204,0.055849


In [35]:
result_atual = result.loc[7]
time_period = result_atual['Time_period']
var_bb = result_atual['var_bb']
pontos = result_atual['pontos']
rate_stop = 1
rate_tp = 2
spread = 0.00005
timeframe = 5


df=df_puro.copy()
bt = setup_tsl(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=False)

stats = bt.run()

stats

Start                     2022-12-12 00:10:00
End                       2023-01-13 23:50:00
Duration                     32 days 23:40:00
Exposure Time [%]                   20.142857
Equity Final [$]             101615673.163391
Equity Peak [$]              102002087.663081
Return [%]                           1.615673
Buy & Hold Return [%]                2.845451
Return (Ann.) [%]                    16.38722
Volatility (Ann.) [%]                3.724141
Sharpe Ratio                         4.400269
Sortino Ratio                       15.375959
Calmar Ratio                        36.715061
Max. Drawdown [%]                   -0.446335
Avg. Drawdown [%]                   -0.074318
Max. Drawdown Duration       10 days 06:45:00
Avg. Drawdown Duration        0 days 17:27:00
# Trades                                  153
Win Rate [%]                        47.712418
Best Trade [%]                       0.361842
Worst Trade [%]                     -0.058062
Avg. Trade [%]                    

In [36]:
bt.plot()

Row(id='2475', ...)

In [37]:
df_trades = stats['_trades']
df_trades['Hora_h'] = df_trades['EntryTime'].apply(lambda x: x.hour)
df_trades['resultado_binario'] = df_trades['ReturnPct'].apply(lambda x: 1 if x > 0 else 0)

print(f'Time_period: {time_period} -- var_bb: {var_bb} -- pontos: {pontos}--Rate_tp: {rate_tp}-- Retorno: {round(retorno,2)} -- Pontos_total: {Pontos_total}')
df_new_hora = df_trades[['Hora_h','resultado_binario']]
df_new_hora = df_trades.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

#df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)

Time_period: 30.0 -- var_bb: 2.0 -- pontos: 0.001--Rate_tp: 2-- Retorno: 6.14 -- Pontos_total: 0.88425
    Hora_h  qtd_operacoes  taxa_acerto
0        0              2     0.500000
1        1              9     0.333333
2        2              7     0.428571
3        3              9     0.777778
4        4             10     0.600000
5        5              5     0.400000
6        6              4     0.250000
7        7              9     0.444444
8        8              8     0.375000
9        9              7     0.714286
10      10              9     0.444444
11      11              9     0.555556
12      12              3     0.333333
13      13              7     0.428571
14      14              5     0.600000
15      15             11     0.363636
16      16             10     0.700000
17      17              2     0.000000
18      18              1     0.000000
19      19              4     0.750000
20      20              6     0.500000
21      21              5     0.200000
